<div align="center">
<h1 style="font-weight:bold; font-size:24px; color: #004d44; margin-bottom: 5;">Identifying Cyclist Safety in Urban Traffic</h1>
<h1 style="font-weight:bold; font-size:24px; color: #004d44; margin-top: 8px;">Using Computer Vision</h1>
<br>
<p style="font-size:18px; font-weight:bold; margin: 10;">Joelma Rodrigues</p>
<p style="font-size:18px; margin: 10;">B.Sc. in Computing & IT (4ᵗʰ year)</p>
<p style="font-size:18px; margin: 0;">CCT College Dublin</p>
<br>
<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.cct.ie%2Fwp-content%2Fuploads%2FCCT_Logo_New_Aug_17-2.jpg&f=1&nofb=1&ipt=86bc4d018505dabaf623bb8b28495f926a4cb4ed2cb2ef7cc51583ac6630d2fa" width="450" alt="CCT College Dublin Logo"/>
<br>
<p style="font-size:18px; margin: 5;">May 2025</p>
<p style="font-size:18px; margin: 0;">Supervisor: Dr. Muhammad Iqbal</p>
</div>

<h2 style="color: #026658; font-size: 32px; font-weight:bold;">Part II</h2>

<h2 style="color: #026658;font-size: 32px;">Install Libraries</h2>

In [ ]:
# Libraries to download
#!pip install --upgrade pip
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install opencv-python pillow matplotlib seaborn albumentations ultralytics optuna joblib

In [9]:
# ========== General Utilities ==========
import os                                # Work with folders and file paths
from pathlib import Path                 # Handle folders/files too
import random                            # Pick random items (like images or values)
from collections import Counter          # Count elements 
from PIL import Image                    # Open and plot images (https://pillow.readthedocs.io/)

# ========== Data Handling ==========
import pandas as pd                      # Work with tables and CSV files (https://pandas.pydata.org/)

# ========== Visualisation ==========
import matplotlib.pyplot as plt          # Create simple plots (https://matplotlib.org/)
import seaborn as sns                    # Charts too - (https://seaborn.pydata.org/)
from IPython.display import display      # Display images (https://ipython.readthedocs.io/)
import cv2                               # Image processing (https://docs.opencv.org/)

# ========== Model (YOLOv8) ==========
from ultralytics import YOLO             # YOLO models (https://docs.ultralytics.com/)

<h1 style="color: #044b42; font-size: 36px; font-weight:bold;">Modeling</h1>

<h2 style="color: #026658; font-size: 32px">YOLOv8n Architecture</h2>

In [ ]:
# download pretrained Model YOLOv8n (Nano)
model = YOLO("yolov8n.pt")

In [12]:
# Check the model architectures summary
model.info(verbose=True)

Model summary: 129 layers, 3,011,628 parameters, 0 gradients, 8.2 GFLOPs


(129, 3011628, 0, 8.197376)

<h2 style="color: #026658; font-size: 32px;">Visualise</h2>

<p style="font-size: 16px;">
Netron is an open-source tool developed to visualise neural network architectures (Roeder, 2021). The YOLOv8 network used in this project was exported as yolov8n.onnx.<br>  
Available at: <a href="https://netron.app">https://netron.app</a> at the end of the output.
</p>

In [6]:
# Check the model architectures using summary
model.export(format="onnx")

Ultralytics 8.3.141  Python-3.10.7 torch-2.7.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.53', 'onnxruntime'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 27.7 MB/s eta 0:00:00
   ---------------------------------------- 12.7/12.7 MB 46.7 MB/s eta 0:00:00
   ---------------------------------------- 8/8 [onnxruntime]

requirements: AutoUpdate success  12.3s
WARNING requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.53...
ON

'yolov8n.onnx'

<h2 style="color: #026658; font-size: 32px">Final Model</h2>

<div style="font-size: 16px;">
    <p>The models were trained under a supervised learning approach with a customised dataset containing four classes: bicycle, helmet, scooter and vest. 
    <br>The dataset is customised to the YOLO format. 
    <br>The first training session was executed on a local machine (RTX 2060, 64GB RAM, 6GB VRAM, Intel Core(TM) i7-10750H 2.60GHz).
    <br>Due to time restrictions, the following models were trained on a cloud instance (RTX A6000, 50GB RAM, 48GB VRAM, 9 vCPU). 
    <br>This reduced the training time by approximately 68%.</p>
</div>

In [ ]:
# Training configuration
model.train(
    data="/workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/data.yaml",  # Path to dataset YAML
    epochs=200,                       # Number of training epochs
    batch=128,                        # Batch size according to GPU memory
    imgsz=640,                        # Input image size
    patience=10,                      # Early stopping patience
    device=0,                         # or 'cuda:0'
    name="ciclovia_final_model",      # Name of the model
    project="ciclovia",               # Main project folder inside -> runs

    # ========= Optimization settings =========
    optimizer="SGD",          # Stochastic Gradient Descent optimizer for stable convergence
    lr0=0.01,                 # Initial learning rate
    lrf=0.01,                 # Final learning rate (scheduler end point)
    momentum=0.937,           # Momentum factor for SGD
    weight_decay=0.0005,      # L2 regularization (Ridge Regression)

    # ========= Loss function weights =========
    box=7.0,                  # Box loss weight
    cls=0.8,                  # Classification loss weight (increased: boost underperforming classes)
    dfl=1.5,                  # Distribution focal loss weight

    # ========= Training performance settings =========
    amp=True,                 # Automatic mixed precision for speed
    pretrained=True,          # Use pretrained weights
    workers=8,
    plots=True,               # Save training plots (loss, metrics, etc.)
    verbose=True              # Print detailed logs during training
)

Ultralytics 8.3.140 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX A6000, 45515MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.8, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ciclovia_final_model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=Tru

train: Scanning /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/labels.cache... 32121 images, 0 backgrounds, 1 corrupt: 100%|██████████| 32121/32121 [00:00<?, ?it/s]

train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_13954.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_13955.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_13957.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_13984.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_14029.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_14086.jpg: 1 duplicate labels removed
train: /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/train/images/2-scooter_scooter_14093.jpg: 1 duplicate labels removed
train: /workspace/ca

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.2±0.1 ms, read: 53.5±40.6 MB/s, size: 189.6 KB)


val: Scanning /workspace/ca2-90-JoelmaRodrigues2023246/datasets/ciclovia-yolo/val/labels.cache... 4015 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4015/4015 [00:00<?, ?it/s]


...

                   all       4015       7600      0.756      0.788      0.813      0.558
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 60, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



70 epochs completed in 2.195 hours.
Optimizer stripped from ciclovia/ciclovia_final_model2/weights/last.pt, 6.2MB
Optimizer stripped from ciclovia/ciclovia_final_model2/weights/best.pt, 6.2MB

Validating ciclovia/ciclovia_final_model2/weights/best.pt...
Ultralytics 8.3.140 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA RTX A6000, 45515MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:13<00:00,  1.17it/s]


                   all       4015       7600      0.749      0.794      0.815      0.559
               bicycle        907       2150      0.623      0.625      0.648      0.418
                helmet        939       2461      0.651      0.698      0.736      0.426
               scooter       1430       1796      0.851      0.904      0.924       0.68
                  vest        739       1193      0.873      0.949      0.951      0.711
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to ciclovia/ciclovia_final_model2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dc2d8a585b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

<h1 style="color: #044b42; font-size: 36px; font-weight:bold;">Evaluation</h1>

<h2 style="color: #026658;font-size: 32px;">Test Subset</h2>
<p style="font-size: 16px;"> The metrics were generated via the test subset to evaluate if the model meets the business criteria (mAP@50 ≥ 75%). However, other metrics are fundamental for identifying the performance of the final multi-class object detection model.</p>

In [ ]:
# load the final model
final_model = YOLO("ciclovia/final_model/weights/best.pt")

In [16]:
# Test the final model on the test set
# Use .val() to evaluate the model on the test set
test_final_model = final_model.val(  
    data="datasets/ciclovia-yolo/data.yaml",  # path to data.yaml
    split="test",            # use test set
    imgsz=640,               # image size
    device=0,                # use GPU  
    save_txt=False,           # no need save txt (labelled data)
    project="ciclovia/test",  # folder to save
    name="test_final_model"   # subfolder name
)

Ultralytics 8.3.141  Python-3.10.7 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 459.6524.3 MB/s, size: 65.9 KB)


val: Scanning F:\CCT\CCT\Semester 8\Problem Solving for Industry\CA-12-MAY\ca2-90-JoelmaRodrigues2023246\datasets\ciclovia-yolo\test\labels.cache... 4021 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4021/4021 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 252/252 [00:43<00:00,  5.83it/s]


                   all       4021       8174      0.787      0.804      0.827      0.603
               bicycle        909       2467      0.621      0.654      0.649      0.411
                helmet        941       2388       0.66      0.688      0.719      0.404
               scooter       1431       1579      0.942       0.95      0.972      0.782
                  vest        740       1740      0.925      0.922      0.969      0.816
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to ciclovia\test\test_final_model


<h2 style="color: #026658;font-size: 32px;">Metrics</h2>
<p style="font-size: 16px;"> This shows how the model performs under different thresholds. Highlights the difference between classes. In comparison, the blue and orange lines (bicycle, helmet) had their curve more to the centre, less precise and low recall. The green and red lines (scooter, vest) have curves closer to the upper right corner, indicating better balance over precision and recall.</p>

![Confusion Matrix](ciclovia/test/test_final_model/confusion_matrix_normalized.png)

<div style="font-size: 16px;">
    <h3 style="font-size:18px;  color: #026658;"><strong>Observations</strong></h3>
    <p style="font-size:16px;">After using .val() on the validation set. We can see the model performed well on new data. An overall mAP50 was 82,7%, however, looking at the same metric individually, we can see the classes helmet and bicycle were lower than the others. Specially looking at the mAP50-95 metrics. 
    <br>Further investigations, looking at the normalised Confusion Matrix: the model made several mistakes detecting 51% of the bicycles and 39% of the helmets as background.
    <br> So, the model is good at detecting vest and scooters. But the classes bicycle and helmet can improve their performance.
    </p>
</div>

<h1 style="color: #044b42; font-size: 36px; font-weight:bold;">Deployment</h1>

In [56]:
final_model = YOLO("ciclovia/final_model/weights/best.pt")
# Use .predict() to test the model on a video
final_model.predict(
    source="videos/scooter.mp4",       # path to video
    save=True,                             # save output video
    imgsz=640,                                       # image size
    device=0,                                        # use GPU
    save_txt=False,                                  # no txt output
    project="videos/predict",         # folder to save
    exist_ok=True,                        # overwrite if already exists
)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/108) f:\CCT\CCT\Semester 8\Problem Solving for Industry\CA-12-MAY\ca2-90-JoelmaRodrigues2023246\videos\scooter.mp4: 384x640 (no detections), 12.4ms
video 1/1 (frame 2/108) f:\CCT\CCT\Semester 8\Problem Solving for Industry\CA-12-MAY\ca2-90-JoelmaRodrigues2023246\videos\scooter.mp4: 384x640 (no detections), 11.9ms
video 1/1 (frame 3/108) f:\CCT\CCT\Semester 8\Problem Solving for Industry\CA-12-MAY\ca2-90-JoelmaRodrigues2023246\videos\sco

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bicycle', 1: 'helmet', 2: 'scooter', 3: 'vest'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
    

<h2 style="color: #026658;font-size: 32px;">Simulation</h2>

![Simulation](resources/img/vehicle_detection_simulation.jpg)

![Simulation](resources/img/simulation.jpg)

<p style="font-size: 16px;">The simulation reflected the model's performance evaluated previously. The easier object to detect was the scooter, followed by the vest when closer to the camera. The bicycles were detected with low precision, and the helmet was detected only in one specific scenario with the lowest confidence.</p>

---

<h1 style="font-size:32px; color: #026658;"><strong>References</strong></h1>

<p style="font-size:16px;">
Open Images Dataset, 2022. Open Images V7. [online] Google AI. Available at: 
<a href="https://storage.googleapis.com/openimages/web/index.html">https://storage.googleapis.com/openimages/web/index.html </a>[Accessed 5 March 2025].
</p>

<p style="font-size:16px;">
Roboflow, 2020. Roboflow: Annotate, manage, and train computer vision models. [online] Available at:
<a href="https://universe.roboflow.com/">https://universe.roboflow.com/</a> [Accessed 8 March 2025].
</p>

<p style="font-size:16px;">
Géron, A., 2019. <em>Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems</em>. 3rd ed. Sebastopol: O’Reilly Media.
</p>

<p style="font-size:16px;">
Buslaev, A., Iglovikov, V.I., Khvedchenya, E., Parinov, A., Druzhinin, M. and Kalinin, A.A., 2020. Albumentations: Fast and flexible image augmentations. <em>Information</em>, 11(2), p.125. Available at: 
<a href="https://doi.org/10.3390/info11020125">https://doi.org/10.3390/info11020125</a>
</p>

<p style="font-size:16px;">
Roeder, L., 2021. Netron: Visualizer for neural network, deep learning and machine learning models. Available at: <a href="https://doi.org/10.3390/info11020125">https://netron.app</a> [Accessed 8 May 2025].
</p>

<p style="font-size:16px;">
Ultralytics, 2025. Performance Metrics Deep Dive. Available at: <a href="https://docs.ultralytics.com/guides/yolo-performance-metrics/ ">https://docs.ultralytics.com/guides/yolo-performance-metrics/ </a> [Accessed 12 May 2025].
</p>